In [1]:
from scipy.sparse.linalg import svds
from scipy.sparse import coo_matrix
from scipy.linalg import svd
import numpy as np

In [2]:
archive_title_npz = np.load('movie_titles.npz')
archive_movie_npz= np.load('movie_recommend.npz')
print(dir(archive_title_npz))

['__abstractmethods__', '__class__', '__class_getitem__', '__contains__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_files', 'allow_pickle', 'close', 'f', 'fid', 'files', 'get', 'items', 'iteritems', 'iterkeys', 'keys', 'pickle_kwargs', 'values', 'zip']


In [3]:
archive_title_npz.files

['titles']

In [4]:
archive_title_npz["titles"]

array(["One Flew Over the Cuckoo's Nest (1975)",
       'James and the Giant Peach (1996)', 'My Fair Lady (1964)', ...,
       'Far and Away (1992)', 'Texas Chainsaw Massacre, The (1974)',
       'Contact (1997)'], dtype='<U75')

In [5]:
archive_movie_npz.files

['format', 'shape', 'data', 'row', 'col']

In [6]:
print(dir(archive_movie_npz))

['__abstractmethods__', '__class__', '__class_getitem__', '__contains__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_files', 'allow_pickle', 'close', 'f', 'fid', 'files', 'get', 'items', 'iteritems', 'iterkeys', 'keys', 'pickle_kwargs', 'values', 'zip']


In [7]:
print(archive_movie_npz["data"].shape)
archive_movie_npz["row"].max()
#archive_movie_npz["col"].shape
print(archive_movie_npz["shape"])
print(archive_movie_npz["row"])
print(archive_movie_npz["col"])

(1000209,)
[6040 3706]
[   0    0    0 ... 6039 6039 6039]
[   0   47  144 ... 3493 3508 3575]


In [8]:
archive_title_npz["titles"].shape

(3706,)

In [9]:
archive_movie_npz["shape"]

array([6040, 3706])

In [10]:
len(archive_movie_npz["data"])

1000209

In [13]:

# create matrix of users and movies
matrix = coo_matrix((archive_movie_npz["data"], (archive_movie_npz["row"], archive_movie_npz["col"])))
matrix = matrix.toarray()

# perform SVD on the matrix
U, s, Vt = svd(matrix)
U=U[:,:51]
S=np.diag(s[:51])
Vt=Vt[:51,:]

# reconstruct the original matrix
matrix_reconstructed = np.dot(np.dot(U, S), Vt)
   

In [14]:
def predict_users(user_ids):
    recommended_movies = []
    for user_id in user_ids:
        user_ratings = matrix_reconstructed[user_id, :]
        predicted_ratings = U[user_id] @ S @ Vt

        # find unrated movies and recommend the top-rated ones
        rated_movies = matrix[user_id, :].nonzero()[0]
        unrated_movies = np.setdiff1d(np.arange(matrix.shape[1]), rated_movies)
        sorted_indexes = np.argsort(predicted_ratings[unrated_movies])[::-1]
        top_movies = unrated_movies[sorted_indexes][:5]

        # add the recommended movies to the list
        recommended_movies.append({
            'user_id': user_id,
            'movies': [
                {
                    'title': archive_title_npz["titles"][movie_id],
                    'predicted_rating': predicted_ratings[movie_id]
                }
                for movie_id in top_movies
            ]
        })

    return recommended_movies
predict_users([1,2,3])



[{'user_id': 1,
  'movies': [{'title': 'Men in Black (1997)', 'predicted_rating': 3.385721},
   {'title': 'Philadelphia (1993)', 'predicted_rating': 3.0277863},
   {'title': 'Splash (1984)', 'predicted_rating': 2.7904305},
   {'title': 'Thing From Another World, The (1951)',
    'predicted_rating': 2.6542416},
   {'title': 'Die Hard 2 (1990)', 'predicted_rating': 2.6136901}]},
 {'user_id': 2,
  'movies': [{'title': 'Raiders of the Lost Ark (1981)',
    'predicted_rating': 1.9745914},
   {'title': 'Daylight (1996)', 'predicted_rating': 1.9619726},
   {'title': 'Final Conflict, The (a.k.a. Omen III: The Final Conflict) (1981)',
    'predicted_rating': 1.9602987},
   {'title': 'Princess Bride, The (1987)', 'predicted_rating': 1.9274876},
   {'title': "One Flew Over the Cuckoo's Nest (1975)",
    'predicted_rating': 1.8797325}]},
 {'user_id': 3,
  'movies': [{'title': 'Happiest Millionaire, The (1967)',
    'predicted_rating': 2.1547785},
   {'title': "On Her Majesty's Secret Service (1969